In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

from sklearn import ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical
from keras import models
from keras import layers

In [ ]:
%%capture
!wget https://www.dropbox.com/scl/fi/rz8xmydm1h3bpgjjm30jd/bank_train.csv?rlkey=rylj5xs1icbfokyabtnqo5gdv&dl=0
!wget https://www.dropbox.com/scl/fi/05p2v6pw76tjf7c21rcyb/bank_valid.csv?rlkey=ifbgda9mj7ybbgl2txnzjq9g1&dl=0

In [ ]:
df = pd.read_csv('bank_train.csv', encoding='iso-8859-1')
del df["Unnamed: 0"]
df.isnull().any()

<ipython-input-3-70631bddee46>:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('bank_train.csv', encoding='iso-8859-1')


Surname               False
CreditScore           False
Age                   False
Tenure                False
Balance               False
NumOfProducts         False
HasCrCard             False
IsActiveMember        False
EstimatedSalary       False
Exited                False
Surname_tfidf_0       False
Surname_tfidf_1       False
Surname_tfidf_2       False
Surname_tfidf_3       False
Surname_tfidf_4       False
France                False
Germany               False
Spain                 False
Female                False
Male                  False
Mem__no__Products     False
Cred_Bal_Sal          False
Bal_sal               False
Tenure_Age            False
Age_Tenure_product    False
dtype: bool

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157525 entries, 0 to 157524
Data columns (total 25 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Surname             157525 non-null  int64  
 1   CreditScore         157525 non-null  float64
 2   Age                 157525 non-null  float64
 3   Tenure              157525 non-null  float64
 4   Balance             157525 non-null  float64
 5   NumOfProducts       157525 non-null  float64
 6   HasCrCard           157525 non-null  int64  
 7   IsActiveMember      157525 non-null  int64  
 8   EstimatedSalary     157525 non-null  object 
 9   Exited              157525 non-null  int64  
 10  Surname_tfidf_0     157525 non-null  object 
 11  Surname_tfidf_1     157525 non-null  object 
 12  Surname_tfidf_2     157525 non-null  object 
 13  Surname_tfidf_3     157525 non-null  object 
 14  Surname_tfidf_4     157525 non-null  object 
 15  France              157525 non-nul

In [ ]:
df.head()

,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,...,France,Germany,Spain,Female,Male,Mem__no__Products,Cred_Bal_Sal,Bal_sal,Tenure_Age,Age_Tenure_product
0,2061,634.0,38.0,4.0,0.00,2.0,0,0,145537.57,0,...,1,0,0,1,0,0.0,0.0,0.0,"10.526.315.789.473.600,00",152.0
1,2100,602.0,52.0,1.0,0.00,2.0,1,1,65413.38,0,...,1,0,0,1,0,2.0,0.0,0.0,"19.230.769.230.769.200,00",52.0
2,1030,850.0,69.0,3.0,71467.33,2.0,1,1,143586.25,0,...,0,1,0,1,0,2.0,"4.230.713.630.309.300,00","49.773.101.533.050.600,00","43.478.260.869.565.200,00",207.0
3,1864,775.0,37.0,7.0,0.00,1.0,1,1,38252.25,0,...,0,0,1,0,1,1.0,0.0,0.0,"1.891.891.891.891.890,00",259.0
4,868,823.0,42.0,6.0,87637.28,1.0,0,1,198446.91,0,...,1,0,0,1,0,1.0,"36.344.975.812.422.500,00","4.416.157.449.869.080,00","14.285.714.285.714.200,00",252.0


In [ ]:
names = ['Surname_tfidf_0', 'Surname_tfidf_1', 'Surname_tfidf_3', 'Surname_tfidf_2', 'Surname_tfidf_4', 'Cred_Bal_Sal', 'Bal_sal', 'Tenure_Age', 'EstimatedSalary']
for name in names:
    df[name] = pd.factorize(df[name])[0]
df.head()

,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,...,France,Germany,Spain,Female,Male,Mem__no__Products,Cred_Bal_Sal,Bal_sal,Tenure_Age,Age_Tenure_product
0,2061,634.0,38.0,4.0,0.00,2.0,0,0,0,0,...,1,0,0,1,0,0.0,0,0,0,152.0
1,2100,602.0,52.0,1.0,0.00,2.0,1,1,1,0,...,1,0,0,1,0,2.0,0,0,1,52.0
2,1030,850.0,69.0,3.0,71467.33,2.0,1,1,2,0,...,0,1,0,1,0,2.0,1,1,2,207.0
3,1864,775.0,37.0,7.0,0.00,1.0,1,1,3,0,...,0,0,1,0,1,1.0,0,0,3,259.0
4,868,823.0,42.0,6.0,87637.28,1.0,0,1,4,0,...,1,0,0,1,0,1.0,2,2,4,252.0


In [ ]:
df_valid = pd.read_csv('bank_valid.csv', encoding='iso-8859-1')
del df_valid["Unnamed: 0"]
df_valid.isnull().any()
names = ['Surname_tfidf_0', 'Surname_tfidf_1', 'Surname_tfidf_3', 'Surname_tfidf_2', 'Surname_tfidf_4', 'Cred_Bal_Sal', 'Bal_sal', 'Tenure_Age', 'EstimatedSalary']
for name in names:
    df_valid[name] = pd.factorize(df_valid[name])[0]
df_valid.head()

,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,...,France,Germany,Spain,Female,Male,Mem__no__Products,Cred_Bal_Sal,Bal_sal,Tenure_Age,Age_Tenure_product
0,537,588.0,36.0,2.0,141461.04,2.0,1,0,0,0,...,1,0,0,0,1,0.0,0,0,0,72.0
1,2024,669.0,33.0,1.0,0.00,2.0,0,1,1,0,...,1,0,0,0,1,2.0,1,1,1,33.0
2,1284,635.0,29.0,5.0,0.00,2.0,0,1,2,0,...,1,0,0,0,1,2.0,1,1,2,145.0
3,1817,597.0,35.0,3.0,0.00,2.0,1,1,3,0,...,0,0,1,1,0,2.0,1,1,3,105.0
4,1335,600.0,37.0,1.0,142663.46,1.0,0,1,4,0,...,1,0,0,0,1,1.0,2,2,4,37.0


In [ ]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 9.1 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


X = df.drop(columns=['Exited', 'Surname_tfidf_0', 'Surname_tfidf_1', 'Surname_tfidf_3', 'Surname_tfidf_2', 'Surname_tfidf_4', 'Cred_Bal_Sal', 'Bal_sal', 'Tenure_Age', 'EstimatedSalary'])
y = df['Exited']
X_valid = df_valid.drop(columns=['Exited', 'Surname_tfidf_0', 'Surname_tfidf_1', 'Surname_tfidf_3', 'Surname_tfidf_2', 'Surname_tfidf_4', 'Cred_Bal_Sal', 'Bal_sal', 'Tenure_Age', 'EstimatedSalary'])
y_valid = df_valid['Exited']


# Разделение на обучающий и тестовый наборы
X_train, y_train = X, y

# Масштабирование признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_valid)

# Обучение модели случайного леса

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Прогнозирование на тестовом наборе
y_pred = model.predict(X_test_scaled)

# Оценка модели
print("Accuracy:", accuracy_score(y_valid, y_pred))
print(classification_report(y_valid, y_pred))

Accuracy: 0.8549391532880077
              precision    recall  f1-score   support

           0       0.88      0.94      0.91     13878
           1       0.70      0.53      0.60      3625

    accuracy                           0.85     17503
   macro avg       0.79      0.73      0.76     17503
weighted avg       0.85      0.85      0.85     17503



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from catboost import CatBoostClassifier
import time

df['Cred_Bal_Sal']= df['CreditScore']*df['Balance']/df['EstimatedSalary']
df['Bal_sal']=df['Balance']/df['EstimatedSalary']
df['Tenure_Age']=df['Tenure']/df['Age']

X = df.drop(columns=['Exited', 'Surname_tfidf_0', 'Surname_tfidf_1', 'Surname_tfidf_3', 'Surname_tfidf_2', 'Surname_tfidf_4', 'Bal_sal', 'Tenure_Age', 'EstimatedSalary'])
y = df['Exited']
X_valid = df_valid.drop(columns=['Exited', 'Surname_tfidf_0', 'Surname_tfidf_1', 'Surname_tfidf_3', 'Surname_tfidf_2', 'Surname_tfidf_4', 'Bal_sal', 'Tenure_Age', 'EstimatedSalary'])
y_valid = df_valid['Exited']


X_train, y_train = X, y
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_valid)

model = CatBoostClassifier(random_state=42, max_depth=6, iterations=400, l2_leaf_reg=3, learning_rate=0.1)
model.fit(X_train_scaled, y_train)


0:	learn: 0.6018788	total: 43.6ms	remaining: 17.4s
1:	learn: 0.5376110	total: 78.4ms	remaining: 15.6s
2:	learn: 0.4900079	total: 111ms	remaining: 14.7s
3:	learn: 0.4530494	total: 147ms	remaining: 14.6s
4:	learn: 0.4284723	total: 184ms	remaining: 14.5s
5:	learn: 0.4071636	total: 218ms	remaining: 14.3s
6:	learn: 0.3912520	total: 256ms	remaining: 14.4s
7:	learn: 0.3786650	total: 289ms	remaining: 14.2s
8:	learn: 0.3677441	total: 325ms	remaining: 14.1s
9:	learn: 0.3600245	total: 359ms	remaining: 14s
10:	learn: 0.3541471	total: 393ms	remaining: 13.9s
11:	learn: 0.3488430	total: 428ms	remaining: 13.8s
12:	learn: 0.3456108	total: 467ms	remaining: 13.9s
13:	learn: 0.3421971	total: 508ms	remaining: 14s
14:	learn: 0.3389651	total: 544ms	remaining: 14s
15:	learn: 0.3366034	total: 579ms	remaining: 13.9s
16:	learn: 0.3346154	total: 612ms	remaining: 13.8s
17:	learn: 0.3330127	total: 650ms	remaining: 13.8s
18:	learn: 0.3316516	total: 703ms	remaining: 14.1s
19:	learn: 0.3305440	total: 738ms	remaining: 

In [ ]:
t1 = time.time()
y_pred = model.predict(X_test_scaled)
t2 = time.time() - t1
print("Prediction time: " + str(t2))
print("Accuracy:", accuracy_score(y_valid, y_pred))
print(classification_report(y_valid, y_pred))

Prediction time: 0.017130374908447266
Accuracy: 0.8651659715477347
              precision    recall  f1-score   support

           0       0.88      0.95      0.92     13878
           1       0.75      0.52      0.62      3625

    accuracy                           0.87     17503
   macro avg       0.82      0.74      0.77     17503
weighted avg       0.86      0.87      0.86     17503



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from catboost import CatBoostClassifier
import time

df = pd.read_csv('bank_train.csv', encoding='iso-8859-1')
del df["Unnamed: 0"]
df.isnull().any()
df_valid = pd.read_csv('bank_valid.csv', encoding='iso-8859-1')
del df_valid["Unnamed: 0"]
df_valid.isnull().any()

df['Cred_Bal_Sal'] = pd.to_numeric(df['Cred_Bal_Sal'], errors='coerce')  # coerce errors will convert non-numeric values to NaN
df['Bal_sal'] = pd.to_numeric(df['Bal_sal'], errors='coerce')
df['Tenure_Age'] = pd.to_numeric(df['Tenure_Age'], errors='coerce')
df['EstimatedSalary'] = pd.to_numeric(df['EstimatedSalary'], errors='coerce')

df_valid['Cred_Bal_Sal'] = pd.to_numeric(df_valid['Cred_Bal_Sal'], errors='coerce')  # coerce errors will convert non-numeric values to NaN
df_valid['Bal_sal'] = pd.to_numeric(df_valid['Bal_sal'], errors='coerce')
df_valid['Tenure_Age'] = pd.to_numeric(df_valid['Tenure_Age'], errors='coerce')
df_valid['EstimatedSalary'] = pd.to_numeric(df_valid['EstimatedSalary'], errors='coerce')

df['Cred_Bal_Sal']= df['CreditScore']*df['Balance']/df['EstimatedSalary']



X = df.drop(columns=['Exited', 'Surname_tfidf_0', 'Surname_tfidf_1', 'Surname_tfidf_3', 'Surname_tfidf_2', 'Surname_tfidf_4'])
y = df['Exited']
X_valid = df_valid.drop(columns=['Exited', 'Surname_tfidf_0', 'Surname_tfidf_1', 'Surname_tfidf_3', 'Surname_tfidf_2', 'Surname_tfidf_4'])
y_valid = df_valid['Exited']


X_train, y_train = X, y
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_valid)

model = CatBoostClassifier(max_depth=6, iterations=400, l2_leaf_reg=3, learning_rate=0.1)
model.fit(X_train_scaled, y_train)


<ipython-input-8-cf96597692fc>:8: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('bank_train.csv', encoding='iso-8859-1')


0:	learn: 0.6028409	total: 150ms	remaining: 59.9s
1:	learn: 0.5360291	total: 254ms	remaining: 50.5s
2:	learn: 0.4879701	total: 411ms	remaining: 54.4s
3:	learn: 0.4519564	total: 575ms	remaining: 56.9s
4:	learn: 0.4236758	total: 641ms	remaining: 50.7s
5:	learn: 0.4030431	total: 682ms	remaining: 44.8s
6:	learn: 0.3878566	total: 718ms	remaining: 40.3s
7:	learn: 0.3753008	total: 755ms	remaining: 37s
8:	learn: 0.3660998	total: 794ms	remaining: 34.5s
9:	learn: 0.3585057	total: 875ms	remaining: 34.1s
10:	learn: 0.3521850	total: 965ms	remaining: 34.1s
11:	learn: 0.3473465	total: 1.05s	remaining: 34.1s
12:	learn: 0.3436335	total: 1.14s	remaining: 34s
13:	learn: 0.3405591	total: 1.24s	remaining: 34.3s
14:	learn: 0.3378330	total: 1.39s	remaining: 35.7s
15:	learn: 0.3357237	total: 1.52s	remaining: 36.5s
16:	learn: 0.3338989	total: 1.63s	remaining: 36.7s
17:	learn: 0.3322465	total: 1.74s	remaining: 37s
18:	learn: 0.3309499	total: 1.88s	remaining: 37.6s
19:	learn: 0.3299018	total: 1.95s	remaining: 37

In [ ]:
t1 = time.time()
y_pred = model.predict(X_test_scaled)
t2 = time.time() - t1
print("Prediction time: " + str(t2))
print("Accuracy:", accuracy_score(y_valid, y_pred))
print(classification_report(y_valid, y_pred))

Prediction time: 0.019420623779296875
Accuracy: 0.8637376449751472
              precision    recall  f1-score   support

           0       0.89      0.95      0.92     13878
           1       0.73      0.54      0.62      3625

    accuracy                           0.86     17503
   macro avg       0.81      0.74      0.77     17503
weighted avg       0.86      0.86      0.86     17503

